## eval

In [29]:
def judge(requirement,input_mol_value,output_mol_value):
    if requirement == 'increase':
        return (output_mol_value > input_mol_value)
    if requirement == 'decrease':
        return (output_mol_value < input_mol_value)
    if requirement == 'the same':        
        return (input_mol_value == output_mol_value)
    metas = requirement.split(',')
    if len(metas) == 2: # examples:  'increase, >=2' ,'decrease, >=2'
        direction = metas[0]
        symbol,number =  extract_symbol_and_number(metas[1])
        if direction == 'increase':
            diff = output_mol_value - input_mol_value
        elif direction == 'decrease':
            diff = input_mol_value - output_mol_value
        else:
            raise NotImplementedError
        if symbol == '>=':
            return diff >= float(number)
        elif symbol == '>':
            return diff > float(number)
        elif symbol == '<=':
            return diff <= float(number)
        elif symbol == '<':
            return diff < float(number)
        elif symbol == '==':
            return diff == float(number)
        else:
            raise NotImplementedError
    if len(metas) == 3:
        assert metas[0] == 'range' # only support range currently, e.g. 'range, 7, 8'
        a, b = [float(x) for x in requirement.split(',')[1:]]
        return (a<=output_mol_value<=b)
print(judge('increase, >=2', 5, 8))  # True
print(judge('decrease, >=2', 8, 5))  # True
print(judge('decrease, >=2', 8, 6))  # True
print(judge('the same', 5, 5))       # True
print(judge('range, 7, 8', 5, 7))    # True
print(judge('range, 7, 9.5', 5, 9))  # True
print(judge('decrease, <=2', 8, 7))  # True
print(judge('increase, <=2', 7, 8))  # True
print(judge('increase, ==2', 7, 9))  # True
print(judge('decrease, ==2', 9, 7))  # True
print(judge('decrease', 9, 7))  # True
print(judge('increase', 5, 7))  # True
print('======================================')
print(judge('range, 7, 8', 5, 9))    # False
print(judge('decrease, >=2', 8, 7))  # False
print(judge('increase, <=2', 7, 9.1))  # False
print(judge('decrease, ==2', 9, 8))  # False
print(judge('increase, ==2', 8, 9))  # False
print(judge('increase', 9, 7))  # True
print(judge('decrease', 5, 7))  # True

True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False


In [5]:
import json
with open('/home/v-nianran/src/MOLLM/data/uniform150_test.json', 'r') as json_file:
    dataset= json.load(json_file)
dataset['requirements'][:2]

[{'qed_requ': {'source_smiles': 'O=C([C@H]1CCCC[C@H]1N1CCCC1=O)N1CC2(CC(F)C2)C1',
   'reference_smiles': 'NNC(=O)C(=O)NC1CC2(C1)CN(C(=O)[C@H]1CCCC[C@H]1N1CCCC1=O)C2',
   'property': 'QED',
   'requirement': 'decrease'},
  'logp_requ': {'source_smiles': 'CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1',
   'reference_smiles': 'COc1ccc([C@@H](O)C(=O)N[C@H]2[C@@H]3COC[C@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1',
   'property': 'logP',
   'requirement': 'range, 2, 3'},
  'donor_requ': {'source_smiles': 'O=C(NC[C@H]1CCOc2ccccc21)c1ccc(F)c(C(F)(F)F)c1',
   'reference_smiles': 'CC(C)C[NH+](CC(=O)[O-])C(F)(F)c1cc(C(=O)NC[C@H]2CCOc3ccccc32)ccc1F',
   'property': 'donor',
   'requirement': 'increase'}},
 {'qed_requ': {'source_smiles': 'CC(C)c1cccc(N(C)CCN(C)C(=O)CCNc2cc(-c3ccsc3)cc(-c3cccc(O)c3)n2)c1',
   'reference_smiles': 'CN(C)CCN(C)C(=O)CCNc1cc(-c2ccsc2)cc(-c2cccc(O)c2)n1',
   'property': 'QED',
   'requirement': 'increase, >=0.1'},
  'logp_requ': {'source_smiles'

In [ ]:
import matplotlib.pyplot as plt

'''# Initial population data
initial_qed = [i.fitness[0] for i in init_population]
initial_sa = [i.logp for i in init_population]
initial_donor = [i.donor_num for i in init_population]

# Plot initial population
plt.scatter(initial_qed, initial_sa, color='blue', label='Initial Population')

# Plot initial population points with donor_num > 2 using a star marker
for i in range(len(init_population)):
    if initial_donor[i] > 2:
        plt.scatter(initial_qed[i], initial_sa[i], color='yellow', marker='*')'''

# Final population data
population = final_pops[0]
final_qed = [i.fitness[0] for i in population]
final_sa = [i.logp for i in population]
final_donor = [i.donor_num for i in population]

# Plot final population
plt.scatter(final_qed, final_sa, color='red', label='Final Population')

# Plot final population points with donor_num > 2 using a star marker
for i in range(len(population)):
    if final_donor[i] > 2:
        plt.scatter(final_qed[i], final_sa[i], color='yellow', marker='*')

# Labels and legend
plt.xlabel('QED')
plt.ylabel('Normalized SA')
plt.legend()
plt.title('Initial and Final Populations in QED vs. Normalized SA (without LLM, random generating new candidates)')
plt.show()

In [26]:
import re
def extract_symbol_and_number(requirement_str):
    pattern = r'([<>]=?|==|!=)\s*(-?\d+(\.\d+)?)'
    match = re.search(pattern, requirement_str.strip())
    
    if match:
        symbol = match.group(1)  # Extracts the comparison symbol (e.g., '>=', '==', '!=', etc.)
        number = float(match.group(2))  # Extracts the number (as a float, e.g., 2, -3, 3.14, etc.)
        return symbol, number
    else:
        print('The format of requirement is wrong:', requirement_str)
        raise NotImplementedError
def judge(requirement,input_mol_value,output_mol_value):
    metas = requirement.split(',')
    if metas[0] == 'the same':
        return (input_mol_value == output_mol_value)
    if metas[0] == 'equal':
        return (output_mol_value == float(metas[1]))
    if len(metas) < 3 and metas[0] in ['increase','decrease']: # examples:  'increase, >=2' ,'decrease, >=2'
        direction = metas[0]
        if len(metas) == 1:
            symbol,number = '>',0
        elif len(metas) == 2:
            symbol,number =  extract_symbol_and_number(metas[1])
        if direction == 'increase':
            diff = output_mol_value - input_mol_value
        elif direction == 'decrease':
            diff = input_mol_value - output_mol_value
        else:
            raise NotImplementedError
        if symbol == '>=':
            return diff >= float(number)
        elif symbol == '>':
            return diff > float(number)
        elif symbol == '<=':
            return diff <= float(number)
        elif symbol == '<':
            return diff < float(number)
        elif symbol == '==':
            return diff == float(number)
        else:
            raise NotImplementedError
    elif len(metas) == 3:
        assert metas[0] == 'range' # only support range currently, e.g. 'range, 7, 8'
        a, b = [float(x) for x in requirement.split(',')[1:]]
        return (a<=output_mol_value<=b)
print(judge('decrease',5,7))
print(judge('the same',5,7))
print(judge('equal, 3',5,4 ))
print(judge('the same',5,5))
print(judge('decrease',7,5))
print(judge('increase',5,7))
print(judge('equal, 3',5,3 ))

False
False
False
True
True
True
True


In [24]:
import numpy as np
def transform4moo(original_value,value,op,requirement):
        '''
         this means when the requirement is satisfied, if will give 0 (optimal value), other 1, this means when the requirement is satified,
         this objective will not be main objectives to optimizes, this is useful when we only want the object to reach a certain 
         threshold instead of maximizing or minimizing it
        '''
        if op =='similarity':
            return -value
        if op in ['donor']: 
            is_true = judge(requirement,original_value,value)
            if is_true:
                return 0 # satisfied, give the optimal value
            else:
                return 1 # not satisfied, give a bad value
        else:
            if 'range' in requirement:
                a, b = [int(x) for x in requirement.split(',')[1:]]
                mid = (b+a)/2
                return np.clip(abs(value-mid) * 1/ ( (b-a)/2), a_min=0,a_max=1)
            if op in ['logp','logs']:
                value = value/10
            if 'increase' in requirement:
                return -value
            elif 'decrease' in requirement:
                return value
            else:
                print('only support increase or decrease and range for minimizing')
                raise NotImplementedError
print(transform4moo(5,7,'donor','increase, >=2')) # 0
print(transform4moo(5,7,'donor','increase')) # 0
print(transform4moo(5,7,'donor','decrease')) # 1
print(transform4moo(5,7,'donor','increase, >=3')) # 1
print(transform4moo(5,7,'logp','increase, >=3')) # -0.7
print(transform4moo(5,7,'logp','range,5,9')) # 0.0
print(transform4moo(5,7,'qed','range,5,10')) # 0.2
print(transform4moo(5,7,'qed','range,5,10')) # 0.2
print(transform4moo(5,7,'qed','increase')) # -7

0
0
1
1
-0.7
0.0
0.2
0.2
-7


In [6]:
# smarts_filter: equal,0
# logs: increase
# reduction_potential: range, -2.3. -1.3
# sa: decrease

'increase'

## Make new fair dataset

In [44]:
from eval import judge
from rewards.system import RewardingSystem
system = RewardingSystem()
ops = ['smarts_filter','logs','reduction_potential','sa']
mols = [['CCH','CCOCCOCC'],['CCOCCOCC','C[C@@H]1CCN(C(=O)CCCc2ccccc2)C[C@@H]1O']]
system.evaluate(ops,mols)

{'donor': [[3, 1]]}

## Test and Tutorial

In [20]:
mollm = MOLLM()

In [32]:
## test initialization
mol = 'CCH'
mollm.moo.init_mol_dataset()
init_pops = mollm.moo.generate_initial_population(mol,10)
for i in init_pops:
    print(i.value)

Found local copy...
Loading...
Done!


Loading ZINC dataset...
CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1
N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)cc2)cc1
CC[NH+](CC)[C@](C)(CC)[C@H](O)c1cscc1Br
O=C(Nc1nc[nH]n1)c1cccnc1Nc1cccc(F)c1
Cc1c(/C=N/c2cc(Br)ccn2)c(O)n2c(nc3ccccc32)c1C#N
C[C@@H]1CN(C(=O)c2cc(Br)cn2C)CC[C@H]1[NH3+]
CCOc1ccc(OCC)c([C@H]2C(C#N)=C(N)N(c3ccccc3C(F)(F)F)C3=C2C(=O)CCC3)c1
Cc1ccccc1C(=O)N1CCC2(CC1)C[C@H](c1ccccc1)C(=O)N2C
CCCc1cc(NC(=O)CN2C(=O)NC3(CCC(C)CC3)C2=O)n(C)n1
CCH


In [33]:
## test evaluation in MOO
mollm.moo.requirement_meta['donor_num'] = 1
mollm.moo.evaluate_all(init_pops)

In [34]:
## test generation offspring
prompt = 'How can we modify the molecule  to decrease its logP value? and still for this molecule Consider a molecule with the SMILES string . Propose changes that could increase its QED value by at least 0.1 compared to the pre-optimized value to make it more drug-like. and still for this molecule  Support me in transforming the molecule  by incorporating the same hydrogen bond donors. This molecule is <mol>CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)CCl)C1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12</mol>'
offs = mollm.moo.generate_offspring(init_pops, prompt, 2)
offs

In [35]:
for i in offs:
    print(i.value)

CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)C(Cl)CCl)C1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12
CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)C(O)N1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12
CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)C(=O))C1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12
CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(OH)CCl)C1)Nc1nc(-c2ccc(C=N)c(F)c2)cn2ccnc12


In [37]:
## test selection next generation
next_gen = mollm.moo.select_next_population(init_pops, offs, 3)
for i in next_gen:
    print(i.scores)

[-0.82714972]
[-0.82237984]
[-0.81871365]


In [ ]:
## overall 
mollm.moo.run

## make dataset

In [15]:
import pandas as pd
donor_df = pd.read_csv('/home/v-nianran/src/data/dpo_donor_test.csv')
qed_df = pd.read_csv('/home/v-nianran/src/data/dpo_qed_test.csv')
logp_df = pd.read_csv('/home/v-nianran/src/data/dpo_logp_test.csv')

def extract_smiles_from_string(text):
    pattern = r"<mol>(.*?)</mol>"
    smiles_list = re.findall(pattern, text)
    return smiles_list
num = 50
df = pd.concat([logp_df.iloc[:num],qed_df.iloc[num:num*2],donor_df.iloc[num*2:num*3]])
moles = [extract_smiles_from_string(p)[0] for p in df.prompt]
moles

['CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1',
 'CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)CCl)C1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12',
 'CCOc1cc(C)ccc1Cn1c(-c2cccc(C)n2)nnc1N1CCN(S(C)(=O)=O)[C@@H](C)C1',
 'CCC1Cc2c(nn(-c3ccc4ccccc4c3)c2-c2cc(Cl)cc(Cl)c2)C(CC)N1C(=O)c1cccc(-n2cnnc2)c1',
 'CC1Oc2ccc(-c3cn4ccsc4n3)cc2N(C(Cc2cccs2)C(CCl)C(=O)c2ccc(-c3ccccc3)cc2)C1=O',
 'Cc1cccc(C2CCCCCN2S(=O)(=O)c2cccc3nsnc23)c1',
 'CCC(C)=C(CC)CCC(=O)NC1CCN(C2CCN(C(=O)[C@H]3CC[C@H](C(=O)OC)C3)CC2)CC1',
 'CCCc1ccc(-c2ccc([C@H](O)C[C@H]3CCCN3C[C@@H]3CCC4(CCCC4)O3)cc2)s1',
 'CC(C)(C)c1nc([NH2+]c2nc(Nc3ccc4c(c3)CNCC4(C)C)ncc2C(O)NC2CC2)cs1',
 'CCc1cc2c(CN3CCN(C(=O)c4cccs4)CC3)cc(=O)oc2cc1-c1c(C)cc(C)cc1C',
 'COc1cc(C(=O)OCCN2CCO[C@H](C)C2)ccc1[N+](=O)[O-]',
 'CCOc1ccccc1CCCNC(=O)c1ccccc1SCc1ccccc1',
 'O=C(CCc1ccc[nH]1)N1CC[C@]2(C1)CN(C(=O)C1=COCC1)CCN2C(=O)Oc1c(Cl)cc(-c2ccccc2)cc1Cl',
 'O=C(CCCCN1CCCCCC1)c1ccc2nc(Cc3ccccc3)c(=O)n(Cc3ccc(F)cc3)c2n1',
 'C[C@@H](F)CCN1CC[C@

In [16]:
combine_prompt = []
for index,mol in enumerate(moles):
    new_p = re.sub(r'<mol>.*?</mol>', '', logp_df.prompt[index]) + ' and still for this molecule ' +  \
        re.sub(r'<mol>.*?</mol>', '', qed_df.prompt[index]) + ' and still for this molecule  ' +\
        re.sub(r'<mol>.*?</mol>', '', donor_df.prompt[index]) + f' This molecule is <mol>{mol}</mol>'
    combine_prompt.append(new_p)
len(combine_prompt),combine_prompt[1]

(150,
 'How can we modify the molecule  to decrease its logP value? and still for this molecule Consider a molecule with the SMILES string . Propose changes that could increase its QED value by at least 0.1 compared to the pre-optimized value to make it more drug-like. and still for this molecule  Support me in transforming the molecule  by incorporating the same hydrogen bond donors. This molecule is <mol>CC=C(C=CC(=C(C)CC)N1CCOC(C(Cc2cccs2)C(O)CCl)C1)Nc1nc(-c2ccc(C=N)c(N)c2)cn2ccnc12</mol>')

In [17]:
import json
with open('/home/v-nianran/src/data/test_qed.metadata.json','r') as f:
    qed_meta = json.load(f)
with open('/home/v-nianran/src/data/test_logp.metadata.json','r') as f:
    logp_meta = json.load(f)
with open('/home/v-nianran/src/data/test_donor.metadata.json','r') as f:
    donor_meta = json.load(f)
qed_meta[80],logp_meta[80],donor_meta[80]

({'source_smiles': 'CC(Cc1cncc(Br)c1)c1ccc(CC(=O)N2C[C@@H]3COC[C@H](C2)[C@@H]3NC(=O)c2cc3c(n2C)-c2ncccc2OC3)nc1',
  'reference_smiles': 'Cn1c(C(=O)N[C@H]2[C@@H]3COC[C@H]2CN(C(=O)Cc2ccc(O)cn2)C3)cc2c1-c1ncccc1OC2',
  'property': 'QED',
  'requirement': 'increase'},
 {'source_smiles': 'C=C(C)C(c1ccccc1OCc1cccc(C(=O)NC(C)c2cccc(Br)c2)c1)C(C)(F)Br',
  'reference_smiles': 'CC(NC(=O)c1cccc(COc2ccccc2C(N)=O)c1)c1cccc(Br)c1',
  'property': 'logP',
  'requirement': 'range, 4, 5'},
 {'source_smiles': 'CC(C)(C)c1cc(C(N)=[NH+]O)ccc1C(=O)NN1CC(C(=O)OCC(=O)c2ccc(Cl)cc2Cl)CC1=O',
  'reference_smiles': 'CC(C)(C)c1ccc(C(=O)NN2CC(C(=O)OCC(=O)c3ccc(Cl)cc3Cl)CC2=O)cc1',
  'property': 'donor',
  'requirement': 'decrease, >=2'})

In [18]:
data = {
    'prompts':combine_prompt,
    'requirements':[
        {'qed_requ': qed_meta[i],
        'logp_requ': logp_meta[i],
        'donor_requ': donor_meta[i]} for i in range(len(combine_prompt))
    ]
}
json_file_path = 'data/uniform150_test.json'

with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

In [19]:
with open(json_file_path, 'r') as json_file:
    new_data = json.load(json_file,)
new_data['requirements'][0]

{'qed_requ': {'source_smiles': 'O=C([C@H]1CCCC[C@H]1N1CCCC1=O)N1CC2(CC(F)C2)C1',
  'reference_smiles': 'NNC(=O)C(=O)NC1CC2(C1)CN(C(=O)[C@H]1CCCC[C@H]1N1CCCC1=O)C2',
  'property': 'QED',
  'requirement': 'decrease'},
 'logp_requ': {'source_smiles': 'CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1',
  'reference_smiles': 'COc1ccc([C@@H](O)C(=O)N[C@H]2[C@@H]3COC[C@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1',
  'property': 'logP',
  'requirement': 'range, 2, 3'},
 'donor_requ': {'source_smiles': 'O=C(NC[C@H]1CCOc2ccccc21)c1ccc(F)c(C(F)(F)F)c1',
  'reference_smiles': 'CC(C)C[NH+](CC(=O)[O-])C(F)(F)c1cc(C(=O)NC[C@H]2CCOc3ccccc32)ccc1F',
  'property': 'donor',
  'requirement': 'increase'}}

In [54]:
len(new_data['prompts'])

150

## Experiments

In [24]:
mollm = MOLLM('config/base.yaml')
mollm.run()

Found local copy...
Loading...
Done!


Loading ZINC dataset...


  0%|          | 0/3 [00:00<?, ?it/s]

Suggest new molecules based on molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol> and help me decrease the QED value.I have some molecules with their objective values. <mol>CCn1cc(NC(=O)C2CCN(C(=O)CCc3ccccc3Cl)CC2)cn1</mol>,qed:0.8245023098421231,  
<mol>COC(=O)[C@@H](c1ccccc1Cl)N1CCN(C(=O)CCc2ccccc2Cl)CC1</mol>,qed:0.6444932819174114,  
 Give me two new molecules that are different from all points above, and not dominated by any of the above. You can do it by applying crossover on the points I give to you. Please note when you try to achieving these objectives, the molecules given you propose should be similar to the original molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol>. Do not write code. Do not give any explanation. Each output new molecule must start with <mol> and end with </mol> in SIMLE form
Suggest new molecules based on molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=

 33%|███▎      | 1/3 [00:03<00:06,  3.48s/it]

Suggest new molecules based on molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol> and help me decrease the QED value.I have some molecules with their objective values. <mol>CCCCC(CC)COCOc1ccc(C(=O)NC2CCC3C(COc3ccccc3Cl)CCC2C3)cc1</mol>,qed:0.229442435633374,  
<mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol>,qed:0.21636799460509812,  
 Give me two new molecules that are different from all points above, and not dominated by any of the above. You can do it by applying crossover on the points I give to you. Please note when you try to achieving these objectives, the molecules given you propose should be similar to the original molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol>. Do not write code. Do not give any explanation. Each output new molecule must start with <mol> and end with </mol> in SIMLE form
Suggest new molecules based on m

 67%|██████▋   | 2/3 [00:06<00:03,  3.30s/it]

Suggest new molecules based on molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol> and help me decrease the QED value.I have some molecules with their objective values. <mol>CCCCC(CC)COCOc1ccc([C@@H](CO)C(=O)N[C@@H]2CCC3C(COc3ccccc3Cl)CCC2C3)cc1</mol>,qed:0.18847463002625045,  
<mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol>,qed:0.21636799460509812,  
 Give me two new molecules that are different from all points above, and not dominated by any of the above. You can do it by applying crossover on the points I give to you. Please note when you try to achieving these objectives, the molecules given you propose should be similar to the original molecule <mol>CCCCC(CC)COCOc1ccc([C@@H](O)C(=O)N[C@@H]2[C@H]3COC[C@@H]2CN(C(=O)CCc2ccccc2Cl)C3)cc1</mol>. Do not write code. Do not give any explanation. Each output new molecule must start with <mol> and end with </mol> in SIMLE form
Suggest new mol

 67%|██████▋   | 2/3 [00:08<00:04,  4.26s/it]


KeyboardInterrupt: 

### One time experiment

In [10]:
from model.LLM import LLM
from algorithm.base import Item
from MOLLM.algorithm.PromptTemplate import Prompt
import json
from eval import eval_mo_results,mean_sr
from tqdm import tqdm
import re
def extract_smiles_from_string(text):
    pattern = r"<mol>(.*?)</mol>"
    smiles_list = re.findall(pattern, text)
    return smiles_list

class MyPrompt(Prompt):
    def __init__(self,original_mol,requirements,properties):
        super().__init__(original_mol,requirements,properties)
    
    def get_prompt(self):
        requirement_prompt = self.make_requirement_prompt(self.original_mol,self.requirements,self.property)
        instruction_prompt = self.make_instruction_prompt()
        final_prompt = requirement_prompt  + instruction_prompt
        return final_prompt

    def make_instruction_prompt(self):
        prompt = (" Give me five new molecules that are different from all points above, and not dominated by any of the above. "
        "You can do it by applying crossover on the points I give to you. "
        f"Please note when you try to achieving these objectives, the molecules you propose should be similar to the original molecule <mol>{self.original_mol}</mol>. "
        "Do not write code. Do not give any explanation. Each output new molecule must start with <mol> and end with </mol> in SIMLE form"
        )
        return prompt
    
with open('/home/v-nianran/src/MOLLM/data/uniform150_test.json', 'r') as json_file:
    dataset= json.load(json_file)
model = LLM()
exps = [
    ['qed'],
    ['logp'],
    ['donor'],
    ['qed','logp'],
    ['qed','donor'],
    ['logp','donor'],
    ['qed','logp','donor']
]
final_pops = {}
import time
for exp in exps:
    exp_name = '_'.join(exp)
    final_pops[exp_name] = []
    for i in tqdm(range(150)):
        original_mol = extract_smiles_from_string(dataset['prompts'][i])[0]
        requirements = dataset['requirements'][i]
        prompt_gen = MyPrompt(original_mol,requirements,exp)
        prompt = prompt_gen.get_prompt()
        while True:
            try:
                mols = model.chat(prompt)
                break
            except Exception as e:
                print(f'{e}, retry in 60s')
                time.sleep(60)
        mols = extract_smiles_from_string(mols)
        mols = [Item(mol,exp) for mol in mols]
        final_pops[exp_name].append(mols)
        if (i+1)%10 == 0:
            obj = {
                'final_pops':final_pops[exp_name],
            }
            r  = eval_mo_results(dataset,obj,similarity_requ=0.4,ops=exp)
            mean_success_num,mean_success_rate,new_sr = mean_sr(r)
            print(f'experiment {exp}, new mean success rate {new_sr:.4f}, mean success rate: {mean_success_rate:.4f}')

  7%|▋         | 10/150 [00:41<12:42,  5.45s/it]

experiment ['qed'], new mean success rate 0.1800, mean success rate: 0.4000


 13%|█▎        | 20/150 [01:18<12:21,  5.70s/it]

experiment ['qed'], new mean success rate 0.2200, mean success rate: 0.4500


 20%|██        | 30/150 [02:00<14:02,  7.02s/it]

...

experiment ['qed', 'logp', 'donor'], new mean success rate 0.0354, mean success rate: 0.1154


 93%|█████████▎| 140/150 [15:05<03:28, 20.84s/it]

experiment ['qed', 'logp', 'donor'], new mean success rate 0.0343, mean success rate: 0.1143


100%|██████████| 150/150 [16:34<00:00,  6.63s/it]

experiment ['qed', 'logp', 'donor'], new mean success rate 0.0320, mean success rate: 0.1067


In [1]:
from model.LLM import LLM
from algorithm.base import Item
from MOLLM.algorithm.PromptTemplate import Prompt
import json
from eval import eval_mo_results,mean_sr
from tqdm import tqdm
import re
import concurrent.futures
def extract_smiles_from_string(text):
    pattern = r"<mol>(.*?)</mol>"
    smiles_list = re.findall(pattern, text)
    return smiles_list

class MyPrompt(Prompt):
    def __init__(self,original_mol,requirements,properties):
        super().__init__(original_mol,requirements,properties)
    
    def get_prompt(self):
        requirement_prompt = self.make_requirement_prompt(self.original_mol,self.requirements,self.property)
        instruction_prompt = self.make_instruction_prompt()
        final_prompt = requirement_prompt  + instruction_prompt
        return final_prompt

    def make_instruction_prompt(self):
        prompt = (" Give me five new molecules that are different from all points above, and not dominated by any of the above. "
        "You can do it by applying crossover on the points I give to you. "
        f"Please note when you try to achieving these objectives, the molecules you propose should be similar to the original molecule <mol>{self.original_mol}</mol>. "
        "Do not write code. Do not give any explanation. Each output new molecule must start with <mol> and end with </mol> in SIMLE form"
        )
        return prompt
    
with open('/home/v-nianran/src/MOLLM/data/uniform150_test.json', 'r') as json_file:
    dataset= json.load(json_file)
model = LLM()
exps = [
    #['qed'],
    ['logp'],
    ['donor'],
    ['qed','logp'],
    ['qed','donor'],
    ['logp','donor'],
    ['qed','logp','donor']
]
final_pops = {}
import time
for exp in exps:
    exp_name = '_'.join(exp)
    final_pops[exp_name] = []
    for i in tqdm(range(150)):
        original_mol = extract_smiles_from_string(dataset['prompts'][i])[0]
        requirements = dataset['requirements'][i]
        prompt_gen = MyPrompt(original_mol,requirements,exp)
        prompt = prompt_gen.get_prompt()
        while True:
            try:
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    #mols = model.chat(prompt)
                    futures = [executor.submit(model.chat,prompt) for i in range(4)]
                    mols  = ' '.join([future.result() for future in futures])
                    
                break
            except Exception as e:
                print(f'{e}, retry in 60s')
                time.sleep(60)
        mols = extract_smiles_from_string(mols)
        mols = [Item(mol,exp) for mol in mols]
        final_pops[exp_name].append(mols)
        if (i+1)%30 == 0:
            obj = {
                'final_pops':final_pops[exp_name],
            }
            r  = eval_mo_results(dataset,obj,similarity_requ=0.4,ops=exp)
            mean_success_num,mean_success_rate,new_sr = mean_sr(r)
            print(f'experiment {exp}, new mean success rate {new_sr:.4f}, mean success rate: {mean_success_rate:.4f}')

 20%|██        | 30/150 [02:36<17:03,  8.53s/it]

experiment ['logp'], new mean success rate 0.4467, mean success rate: 0.5333


 40%|████      | 60/150 [05:20<18:40, 12.45s/it]

experiment ['logp'], new mean success rate 0.5033, mean success rate: 0.5833


 60%|██████    | 90/150 [08:33<16:04, 16.08s/it]

...

experiment ['qed', 'logp', 'donor'], new mean success rate 0.1267, mean success rate: 0.2667


 80%|████████  | 120/150 [12:32<11:13, 22.45s/it]

experiment ['qed', 'logp', 'donor'], new mean success rate 0.1283, mean success rate: 0.2667


100%|██████████| 150/150 [15:48<00:00,  6.32s/it]

experiment ['qed', 'logp', 'donor'], new mean success rate 0.1147, mean success rate: 0.2467
